In [43]:
import pandas as pd
import numpy as np
import io
import os
# import boto3
from io import StringIO
from pathlib import Path

## GENERAREMOS TRANSFORMACIONES 
streaming_platforms = [
    {"title": "amazon", "file": "amazon_prime_titles.csv"},
    {"title": "disney", "file": "disney_plus_titles.csv"},
    {"title": "hulu", "file": "hulu_titles.csv"},
    {"title": "netflix", "file": "netflix_titles.csv"}
]
cloud_provider = "AWS"
array_pandas = []

def creation_new_rating_dataframe():
    dir = Path("datasets/ratings")
    array_pandas_rating = []
    for f in dir.glob("*.csv"):
       print(f)
       df_auxiliar = pd.read_csv(f) 
       array_pandas_rating.append(df_auxiliar)
    df_rating_completo = pd.concat(array_pandas_rating)
    df_rating_completo["id"] = df_rating_completo["movieId"]
    df_avg = df_rating_completo[["id", "rating"]].groupby(['id']).mean()
    df_avg["score"] = df_avg["rating"].apply(lambda x:round(x,2))
    df_avg = df_avg.drop(columns=['rating'])
    print("Veamos las cosas 123", df_avg.head(10))
    print(df_avg.info())


    # print(df_avg)
    # print("la cantidad de movies df_rating_completo" , df_rating_completo.count())
    # print("la cantidad de movies agrupadas" , df_avg.head(10))
    return df_avg

# def combine_multiples_dataframes(movies , rating):



def generate_dataframe(streaming_plat):

    # Generate id field
    first_char = streaming_plat["title"][0]


    # df_ratings = pd.read_csv( "datasets/ratings/1.csv" )

    df_streaming = pd.read_csv( "datasets/" + streaming_plat["file"] )

    df_streaming["id"] = first_char +  df_streaming["show_id"]

    # Replace the null values in the column rating with the letter G
    df_streaming.loc[df_streaming['rating'].isna(), "rating"] = "G"


    # Change the original date format to AAAA-mm-dd
   
    df_streaming['date_added'] = pd.to_datetime((df_streaming["date_added"]).str.strip(), format='%B %d, %Y').dt.strftime('%Y-%m-%d')

    df_streaming.info()


    # Transform to lowercase
    columnsHeader = list(df_streaming.columns.values)
    # print(df_streaming)
    for x in columnsHeader:
        if x not in ["date_added", "release_year" , "my_version"] :
            # print("Pintando la x",x)
            # print("El elemento es ",  streaming_plat["title"])
            df_streaming[str(x)] = df_streaming[str(x)].astype(str).str.lower()
            # df_streaming.loc[not df_streaming[str(x)].isna(), str(x)] = df_streaming[str(x)].str.lower()



    # Split the column duration in two cases : duration_int and duration_type
    #Divido la columna "duration" en duration_int y duration_type. 
    df_streaming[['duration_int', 'duration_type']] = df_streaming['duration'].astype(str).str.split(pat=' ', expand=True)
    df_streaming.loc[df_streaming['duration_type'] == 'seasons', 'duration_type'] = 'season'
    df_streaming.loc[df_streaming['duration_type'].isna(), 'duration_type'] = None


    # df_streaming.loc[df_streaming[str(x)], str(x)] = df_streaming[str(x)].str.lower()

    

    return df_streaming



# def save_dataframe_in_bucket(cloud_provider, input_dataframe):
#     match cloud_provider:
#         case "AWS":
#             print("Provider AWS")
#             # AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
#             # AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
#             # AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
#             # AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")
#             # AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
#             # AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
#             # AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
#             # AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")
            
#             bucket = 'pruebas-big-data' # already created on S3
#             csv_buffer = StringIO()
#             input_dataframe.to_csv(csv_buffer)
#             s3_resource = boto3.resource('s3')
#             s3_resource.Object(bucket, 'df.csv').put(Body=csv_buffer.getvalue())



#         case "GCP":
#             print("Provider GCP")
#         case "AZURE":
#             print("Provider AZURE")
#         case _:
#             print("Otro proveedor")


    


for x in streaming_platforms:
    array_pandas.append(generate_dataframe(x))


df_completo = pd.concat(array_pandas)

df_rating_movies = creation_new_rating_dataframe()
# print("df completo")
# print(df_completo.info())
# print("df df_rating_movies")
# print(df_rating_movies.info())

final_frame = pd.merge(df_completo, df_rating_movies , on= "id")
# print("Hello estas cosas")
# print(final_frame.info())

final_frame.to_csv("app/db/MOVIES_DB.csv", index=False)
final_frame.to_json("app/db/MOVIES_DB.json", orient="records")






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9668 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
 12  id            9668 non-null   object
dtypes: int64(1), object(12)
memory usage: 982.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       

['df_avg','rating']